# Demo

## Running Sweep

In [ ]:
from resonance_py.measurements.resonatorMeasurement import ResonatorMeasurement
from resonance_py.drivers.keysightN5221B import KeysightN5221B
from resonance_py.drivers.SetAttenuation import set_attenuation
import matplotlib.pyplot as plt
from resonance_py.utils.statistics import comp2magdB

import numpy as np
from datetime import datetime
from pathlib import Path
import os

atten_address = 'TCPIP0::192.168.10.27::inst0::INSTR'
vna_address = "TCPIP0::K-N5221B-11311.local::hislip0::INSTR"

# ================================================
# Connect to the PNA Instrument
# ================================================
pna = KeysightN5221B("pna", vna_address)


In [ ]:
# ================================================
# Run Full Experiment
# ================================================

center_freq = 4.4e9  # 7.5 GHz
span = 870e6           # 1 GHz span
points = 10e3         # 201 measurement points

measurement_settings = {
        # General settings
        "centerFrequency": 4.4e9,       # 7.0 GHz
        "frequencySpan": 870e6 ,         # 1.0 GHz span
        "systemAttenuation": 0,        # 60 dB attenuation in system
        "selectedResonators": [0, 1, 2, 3, 4], # Resonators to be measured
        
        # Resonator settings
        "numOfResonators": 5,           # Expect 3 resonators
        "attenuationValues": [ 40, 35, 30, 20, 10, 5, 0],  # Attenuation values to test

        "is_dip": True,
        "fix_zero_widths": True,
        "fwhm_index_width_limit": 80,
        
        # Sweep Settings
        'survey': {
            'points': 10e3,
            'if_bandwidth': 1000,
            'measurement': 'S21',
            'save_data': True,
        },
        'refine': {
            'initial_span': 1e6,
            'points': 10e3,
            'if_bandwidth': 1000,
            'measurement': 'S21',
            'span_refinement_value': 0.5,  # Factor to reduce span on each iteration (does this make sense or should we be closer to 0.8-0.9?)
            'fwhm_index_width_limit': 80,  # Minimum number of points across FWHM
            'save_data': True,
        },
        'segment': {
            'points': [75, 60, 40, 50],
            'segment_factors': [0.5, 1, 2, 3],  # Width factors for segments
            'measurement': 'S21',
            'averaging': 1,
            'save_data': True,
        }
        }

save_dir = Path.cwd() /'data/'
date_str = datetime.now().strftime("%Y%m%d")
experiment_name = f"32025_Sample1_Cooldown3"
save_path = os.path.join(save_dir, experiment_name)

measurement = ResonatorMeasurement(
        pna=pna,
        save_path=save_path,
        save_base_name=experiment_name,
        settings=measurement_settings,
        atten_address=atten_address
    )

measurement.full_scan()
# measurement.survey_scan()
# Add graphs to survey and refine sweeps

## Fitting

In [ ]:
import re
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilenames
from resonance_py.data_analysis.resonator_data import ResonatorData
from resonance_py.data_analysis.res_iS21 import single_resonator_fit
from pathlib import Path

In [ ]:
measurement.segment_data['resonator'][0]['run_data'][0]

In [ ]:

data = measurement.segment_data['resonator'][0]['run_data'][0]
resonator = data['resonator']
attenuation = data['sweep_settings']['attenuation']
save_to = Path().cwd() / 'data/log_files/32025_Sample1_Cooldown3/plots'
plot_name = f"R0{resonator+1}_atten{attenuation}dB"


save_to_info ={
    "save_plots": True,  # Save plots
    "file_type": "png",  # Default file type for saving results
    "plot_name_format": 'manual',  # Format for saving plots
    "plot_group": "plots",
    "show_plots": True,  # Show plots
    'file_path': save_to,  # Path to save plots
    'plot_name': plot_name,
}

resonator_data = ResonatorData(freq=data['frequencies'],raw_s21=data['s21'])
resonator_data.save_to_info = save_to_info

resonator_data.atten = data['sweep_settings']['attenuation']
resonator_data.fit.update({'model': 'Probst'})

resonator_data = single_resonator_fit(resonator_data, opts={"plot":False})

In [ ]:
import re
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilenames
from resonance_py.data_analysis.resonator_data import ResonatorData
from resonance_py.data_analysis.modeling import single_resonator_fit
from pathlib import Path

def analyze_resonator_data(get_file='gui,', model = 'probst', dir=None, save_to_info=None):
    # Function to analyze resonator data
    # This function will contain the logic to analyze the data
    # and return the results in a structured format.

    def getfiles(method='gui', directory: Path =None):
        if method == 'gui':
            root = Tk()
            Tk().withdraw()  # we don't want a full GUI, so keep the root window from appearing
            filenames = askopenfilenames(  # show an "Open" dialog box and return paths to selected files
                title="Select data files",
                filetypes=[("Text files", "*.txt"), ("All files", "*.*")]
            )

            root.destroy()
            return filenames
        elif method == 'dir':
            filenames = [f for f in directory.iterdir() if re.match(r'241104 D2G0437 R3C4 Q2-2 R(\d{2}-\d{2}dB-)', str(f.name))]
            return filenames
    
    
    rows = []
    filenames = getfiles(method=get_file, directory=dir)

    for filename in filenames:
        
        file_path = Path(filename)
        
        filename = file_path
        
        print(file_path)
        
        save_to = Path(filenames[0]).parent.parent / 'python_plots'
        save_to_info.update({'file_path': save_to})
        plot_name = filename.stem
        save_to_info.update({'plot_name': plot_name})

        resonator_data = ResonatorData.from_file(filename, save_to_info=save_to_info)

        atten = int(re.findall(r'R\d{2}-(\d{2})dB', file_path.name)[0])
        resonator_data.atten = atten
        resonator_data.fit.update({'model': model})

        resonator = re.findall(r'R\d{2}-\d{2}dB',str(file_path.name))[0][:3]
        resonator_data = single_resonator_fit(resonator_data, opts={"plot":False})
        fo = resonator_data.fit['fo']
        Qi = resonator_data.fit['Qi']
        QiError = resonator_data.fit['QiError']
        Qc = resonator_data.fit['Qc']
        QcError = resonator_data.fit['QcError']
        Qt = resonator_data.fit['Qt']
        QtError = resonator_data.fit['QtError']
        phi = resonator_data.fit['phi']
        np_ = resonator_data.Np
        s21max = resonator_data.max_raw_s21
        s21min = resonator_data.min_raw_s21
        iS21CircSNR = resonator_data.is21circ_snr


        rows.append({'resonator_number': resonator,
                        'fo': fo,
                        'Np': np_,
                        'Qi': Qi,
                        'deltaQi': QiError,
                        'Qc': Qc,
                        'deltaQc': QcError,
                        'Qt': Qt,
                        'deltaQt': QtError,
                        'phi': phi,
                        'S21max (dB)': s21max,
                        'S21min (dB)': s21min,
                        'iS21CircSNR': iS21CircSNR,
                        'Data filename': filename})
        


    df = pd.DataFrame(rows)

In [ ]:

save_to_info ={
    "save_plots": False,  # Save plots
    "file_type": "png",  # Default file type for saving results
    "plot_name_format": 'manual',  # Format for saving plots
    "plot_group": "plots",
    "show_plots": True,  # Show plots
}

analyze_resonator_data(get_file='gui', model='Probst' ,dir=None, save_to_info=save_to_info)

resonator_data = ResonatorData.from_file(filename, save_to_info=save_to_info)

atten = int(re.findall(r'R\d{2}-(\d{2})dB', file_path.name)[0])
resonator_data.atten = atten
resonator_data.fit.update({'model': model})

resonator = re.findall(r'R\d{2}-\d{2}dB',str(file_path.name))[0][:3]
resonator_data = single_resonator_fit(resonator_data, opts={"plot":False})

# New VNA

In [ ]:
import pyvisa

In [ ]:
rm = pyvisa.ResourceManager()

In [ ]:
rm.list_resources()

In [ ]:
vna = rm.open_resource('TCPIP0::192.168.10.20::inst0::INSTR')

In [ ]:

vna.query(':FORMat:DATA?')

In [ ]:

data = vna.query_ascii_values(":SENS1:FREQ:DATA?")

In [ ]:
from ResonatorProject.drivers.keysightE5072A import KeysightE5072A
from ResonatorProject.measurements.resonatorMeasurement import ResonatorMeasurement
from ResonatorProject.drivers.keysightN5221B import KeysightN5221B
from ResonatorProject.drivers.SetAttenuation import set_attenuation
import matplotlib.pyplot as plt
from ResonatorProject.utils.statistics import comp2magdB

import numpy as np
from datetime import datetime
from pathlib import Path
import os
address = 'TCPIP0::192.168.10.20::inst0::INSTR'
vna = KeysightE5072A('E5072A', address)

In [ ]:
from ResonatorProject.utils.segmentation import create_resonator_segments
freqs = [4.5e9]
fwhm = 1e6  # 1 MHz FWHM for all resonators
segments_data = create_resonator_segments(freqs, fwhm, fill_gaps=False, f_sec=[.5,3], n_points=[75, 60])
segments_data

In [ ]:
vna.segmented_sweep(segments_data)

In [ ]:
# ================================================
# Run Full Experiment
# ================================================

center_freq = 4.4e9  # 7.5 GHz
span = 870e6           # 1 GHz span
points = 10e3         # 201 measurement points

measurement_settings = {
        # General settings
        "centerFrequency": 6.7e9,       # 7.0 GHz
        "frequencySpan": 1.7e9 ,         # 1.0 GHz span
        "systemAttenuation": 0,        # 60 dB attenuation in system
        "selectedResonators": [0], # Resonators to be measured
        
        # Resonator settings
        "numOfResonators": 1,           # Expect 3 resonators
        "attenuationValues": [ 40, 35, 0],  # Attenuation values to test

        "is_dip": False,
        "fix_zero_widths": True,
        "fwhm_index_width_limit": 80,
        
        # Sweep Settings
        'survey': {
            'points': 10e3,
            'if_bandwidth': 1000,
            'measurement': 'S21',
            'save_data': False,
        },
        'refine': {
            'initial_span': 1e6,
            'points': 10e3,
            'if_bandwidth': 1000,
            'measurement': 'S21',
            'span_refinement_value': 0.5,  # Factor to reduce span on each iteration (does this make sense or should we be closer to 0.8-0.9?)
            'fwhm_index_width_limit': 80,  # Minimum number of points across FWHM
            'save_data': False,
        },
        'segment': {
            'points': [75, 60, 40, 50],
            'segment_factors': [0.5, 1, 2, 3],  # Width factors for segments
            'measurement': 'S21',
            'averaging': 1,
            'save_data': False,
        }
        }

save_dir = Path.cwd() /'data/'
date_str = datetime.now().strftime("%Y%m%d")
experiment_name = f"32025_Sample1_Cooldown3"
save_path = os.path.join(save_dir, experiment_name)

measurement = ResonatorMeasurement(
        pna=vna,
        save_path=save_path,
        save_base_name=experiment_name,
        settings=measurement_settings,
        atten_address=address
    )


# measurement.survey_scan()
# Add graphs to survey and refine sweeps

In [ ]:
refined_data, all_refined = measurement.refinement_scan(0)
segment_data = measurement.segmented_scan(refined_data)

In [ ]:
res_complex_data, res_frequencies = vna.linear_sweep(
    center_frequency=measurement_settings['centerFrequency'], 
    frequency_span=measurement_settings['frequencySpan'], 
    points=measurement_settings['survey']['points'], 
    if_bandwidth=measurement_settings['survey']['if_bandwidth'],
    measurement=measurement_settings['survey']['measurement'],
)

In [ ]:
refine_settings = measurement.settings['refine']

frequencies = measurement.survey_data['frequencies']
refined_center = measurement.survey_data['peaks_info']['fo'][0]
refined_span = refine_settings['initial_span']
refined_if_bandwidth = refine_settings['if_bandwidth']
refined_points = refine_settings['points']

res_complex_data, res_frequencies = vna.linear_sweep(
    center_frequency=refined_center, 
    frequency_span=refined_span, 
    points=refined_points, 
    if_bandwidth=refined_if_bandwidth,
    measurement=refine_settings['measurement']
)

In [ ]:
# 
# vna.auto_sweep(False)
# vna.write(":ABOR")
vna.ask(":SENS1:SWE:TIME?").strip()
# vna.write(":INIT:IMM")

In [ ]:
from time import sleep
# vna.write("*OPC")
# vna.auto_sweep(True)
vna.write("INIT:IMM")
status = int(vna.ask(":STATus:OPERation:CONDition?").strip())
print(status)
while status:
    sleep(10)
    status = int(vna.ask(":STATus:OPERation:CONDition?").strip())
    print(status)

In [ ]:
vna.auto_sweep(False)
if vna.auto_sweep():
    print("Auto sweep is on")

In [ ]:
vna.ask(":STAT:OPER?")